## Functionality that needs to be added

- ~~Trim Raw Input~~
- ~~Night vs Day bouts (critera - choose where over 50% of bout is spent, in case of tie choose night)~~
- ~~Create Sleep Profile~~
- Create summary stats
- Create function to run all flies and arrange in exportable format
- Write to seperate sheets in Excel
- Adapt to python script that can be run in terminal - takes input .csv and exports excel file

### Import Libraries that will be needed

In [1]:
import pandas as pd
import numpy as np
import datetime

### Import data from .csv

In [2]:
df_raw = pd.read_csv("Bethany_Raw.csv")

### Trim to 24 hour range

In [3]:
lower_limit = datetime.time(10,0,0)
upper_limit = datetime.time(10,5,0)
def get_data_range(L, lower_limit, upper_limit):
    """
    Find the start and end indices for a 24 hour range starting between a given time range
    Takes arguments L - a list of datetime.time objects, lower_limit and upper_limit - datetime.time objects 
    to specificy the range in which to choose the start time
    Returns a tuple: (start_index, end_index)
    """
    start_index = 0
    for i in range(len(L)):
        if L[i] >= lower_limit and L[i] < upper_limit:
            start_index = i
            break
    return (start_index, start_index + 288)


In [4]:
# Trim raw data to 24 hour window
df_raw.Time = pd.to_datetime(df_raw.Time, format='%H:%M:%S').dt.time
trim_index = get_data_range(list(df_raw.Time), lower_limit, upper_limit)
df = df_raw[trim_index[0]:trim_index[1]]
df = df.reset_index(drop=True)

### Make a column of bools to indicate day vs night

In [5]:
def isNight(x):
    """
    Take datetime.time object and return True if between 10pm and 10am, False otherwise 
    """
    night_start = datetime.time(22,0,0)
    night_end = datetime.time(10,0,0)
    if (x >= night_start and x < datetime.time(23, 59, 59)) or x < night_end:
        return True
    else:
        return False

In [6]:
# Create Boolean column indicating Night=True Day=False
df.loc[:,'isNight'] = df.Time.apply(isNight)

### Generate bout indices

In [7]:
# Script for collecting bouts
# Will need to create a list of bout indices to access resp data during bouts
def get_bout_indices(L):
    """
    Takes a list L and returns a list of tuples of the start/end indices in which sleeping bouts occurred.
    I.e. if two sleeping bouts occured, the first from index 5 to 20, and the second from index 30 to 40,
    this function will return [(5,20), (30,40)]
    """
    indices = []
    start_index = 1
    end_index = 1
    in_bout = False
    
    for i in range(len(L)):
        if L[i] == 0 and in_bout == False: 
            start_index = i
            in_bout = True
        if (L[i] != 0 or i == len(L)-1) and in_bout == True:
            end_index = i
            in_bout = False
            if i == len(L)-1:
                indices.append((start_index, end_index+1))
            else:
                indices.append((start_index, end_index))
    return indices 
        

### Get day bouts and night bouts

In [8]:
# Need to put this into a function, and make a loop to generate day/night bouts for each fly of 5 in the df
def get_day_night_bouts(df, resp_colnum, sleep_colnum):
    """
    Takes a dataframe, the colnumn number for resp data, column number for sleep data
    Returns a tuple of two lists of lists: resp data for daytime sleep bouts, resp data for nighttime sleep bouts
    """
    bout_indices = get_bout_indices(list(df.iloc[:,sleep_colnum]))
    resp_bouts_day = []
    resp_bouts_night = []
    for i in bout_indices:
        if df.isNight[i[0]:i[1]].mean() >= 0.5:
            resp_bouts_night.append(list(df.iloc[:,resp_colnum])[i[0]:i[1]])
        else:
            resp_bouts_day.append(list(df.iloc[:,resp_colnum])[i[0]:i[1]])
    if not resp_bouts_day:
        resp_bouts_day = ['none']
    if not resp_bouts_night:
        resp_bouts_night = ['none']
    return resp_bouts_day, resp_bouts_night

def get_all_bouts(df):
    """
    Returns a tuple of two lists of lists, the first contains all day bouts, the second contains all night bouts 
    Bouts from different flies are seperated by any empty list - to become empty column in dataframe  
    """
    resp_colnum = 4
    sleep_colnum = 11
    all_day_bouts = []
    all_night_bouts = []
    for i in range(5):
        day_bouts, night_bouts = get_day_night_bouts(df, resp_colnum, sleep_colnum)
        all_day_bouts += day_bouts + [[]]
        all_night_bouts += night_bouts + [[]]
        resp_colnum+=1
        sleep_colnum+=1
    return all_day_bouts, all_night_bouts

In [9]:
# Create dataframes for day/night bouts
all_day_bouts, all_night_bouts = get_all_bouts(df)
all_day_bouts_df = pd.DataFrame(all_day_bouts).transpose()
all_night_bouts_df = pd.DataFrame(all_night_bouts).transpose()

In [10]:
all_day_bouts_df

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,1.039541,1.180896,1.284699,1.227334,1.185590,1.130278,0.986619,NaN,1.098401,NaN,...,1.366439,1.471703,1.284412,NaN,1.592927,1.589793,1.502461,1.616515,1.838532,NaN
1,1.427665,1.383591,1.285172,1.295008,1.095631,1.101771,1.009748,NaN,NaN,NaN,...,1.330130,1.332372,1.294859,NaN,1.610929,NaN,NaN,1.530170,1.890970,NaN
2,NaN,1.294287,NaN,1.206533,1.199461,1.044584,0.936784,NaN,NaN,NaN,...,1.410944,1.315407,1.380678,NaN,1.390133,NaN,NaN,1.537473,NaN,NaN
3,NaN,1.249192,NaN,NaN,NaN,1.026278,0.938204,NaN,NaN,NaN,...,1.311169,1.319408,1.336910,NaN,NaN,NaN,NaN,1.466006,NaN,NaN
4,NaN,1.326397,NaN,NaN,NaN,8.365838,0.951994,NaN,NaN,NaN,...,1.582302,1.280412,1.348364,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,1.331773,NaN,NaN,NaN,0.910308,0.950158,NaN,NaN,NaN,...,1.514232,1.296456,1.371576,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,1.148687,NaN,NaN,NaN,1.019505,0.945871,NaN,NaN,NaN,...,1.406155,1.219918,1.390585,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,1.122283,NaN,NaN,NaN,1.070171,0.945160,NaN,NaN,NaN,...,1.374179,1.174965,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,1.248453,NaN,NaN,NaN,NaN,0.924083,NaN,NaN,NaN,...,1.469105,1.379992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,1.113087,NaN,NaN,NaN,NaN,1.021218,NaN,NaN,NaN,...,NaN,1.277233,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Get sleep profiles

In [11]:
def get_sleep_profile(df, resp_colnum, sleep_colnum):
    """
    Returns a list of tuples with sum of metabolic rate and the percentage sleep for each hour in the 24 hour window
    """
    fly1_sleep = df.iloc[:,sleep_colnum]
    fly1_resp = df.iloc[:,resp_colnum]
    hourly_resp_sleep = []
    for i in range(24):
        sleep = fly1_sleep[i*12:i*12+12]
        resp = fly1_resp[i*12:i*12+12]
        num_sleep_blocks = 0
        for i in sleep:
            if i == 0:
                num_sleep_blocks += 1
        sleep_avg = num_sleep_blocks / 12 * 100
        hourly_resp_sleep.append((resp.sum(), sleep_avg))
    return hourly_resp_sleep

def make_sleep_profile_dict(df):
    """
    Returns a dictionary containing a time index and the sleep profiles (both metabolic rate sum and average sleep)
    for each fly in the dataframe
    """
    resp_colnum = 4
    sleep_colnum = 11
    sleep_profile_dict = {}
    sleep_profile_dict['Time'] = [df.Time[x*12-1] for x in range(1, 25)]
    for i in range(5):
        sleep_profile = get_sleep_profile(df, resp_colnum, sleep_colnum)
        sleep_profile_dict[df.columns[sleep_colnum] + ' MR Sum'] = [x[0] for x in sleep_profile] #name of genotype is header of sleep column
        sleep_profile_dict[df.columns[sleep_colnum] + ' Avg Sleep'] = [x[1] for x in sleep_profile]
        resp_colnum+=1
        sleep_colnum+=1
    return sleep_profile_dict

def make_colnames(df):
    """
    Create list of ordered column names for dataframe to be created from sleep_profile dictionary
    """
    colnames = ['Time']
    for i in range(11,16):
        colnames.append(df.columns[i] + ' MR Sum')
        colnames.append(df.columns[i] + ' Avg Sleep')
    return colnames

In [12]:
#Creat test df for sleep profile
test_dict = make_sleep_profile_dict(df)
test_df = pd.DataFrame(test_dict, columns=make_colnames(df))

In [13]:
test_df

,Time,w1118 BL 6326 MR Sum,w1118 BL 6326 Avg Sleep,w1118 BL 6326.1 MR Sum,w1118 BL 6326.1 Avg Sleep,eaat2 e04636 MR Sum,eaat2 e04636 Avg Sleep,eaat2 e04636.1 MR Sum,eaat2 e04636.1 Avg Sleep,eaat2 e04636.2 MR Sum,eaat2 e04636.2 Avg Sleep
0,10:56:04,13.666723,0.000000,18.943555,8.333333,16.985112,0.000000,18.626518,8.333333,20.879742,0.000000
1,11:56:04,14.268239,0.000000,20.308204,0.000000,16.451909,0.000000,14.862725,83.333333,18.561291,0.000000
2,12:56:04,14.507468,0.000000,20.116906,0.000000,16.343367,0.000000,16.466497,41.666667,18.079725,0.000000
3,13:56:04,15.105396,0.000000,19.625666,0.000000,16.287813,0.000000,17.933557,16.666667,18.222576,0.000000
4,14:56:04,14.410743,0.000000,19.774922,0.000000,16.533504,0.000000,14.094162,100.000000,17.426245,0.000000
5,15:56:04,15.773512,75.000000,18.800395,0.000000,15.486906,0.000000,15.227216,41.666667,17.478851,0.000000
6,16:56:04,14.073482,100.000000,18.979456,0.000000,14.596119,0.000000,16.543512,16.666667,18.212323,25.000000
7,17:56:04,15.142213,66.666667,19.021843,0.000000,17.382679,8.333333,18.153170,58.333333,19.557847,16.666667
8,18:56:04,15.210787,41.666667,19.139298,0.000000,17.182623,0.000000,15.557176,100.000000,20.330804,33.333333
9,19:56:05,20.560867,83.333333,20.722830,0.000000,18.735982,0.000000,13.717040,100.000000,18.886279,0.000000


## Export to .csv

In [14]:
# Now we save this list of lists as a csv
# resp_bouts_df = pd.DataFrame(resp_bouts).transpose()
# resp_bouts_df.to_csv('test.csv', index=False, header=False)